In [ ]:
import google.generativeai as genai

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))


In [2]:
import re
import time
import os
import json
import google.generativeai as genai

# ==============================
# CONFIGURATION
# ==============================

INPUT_FILE = "formal_specifications.json"
OUTPUT_DIR = "verification_rounds"

MODEL = genai.GenerativeModel("gemini-2.5-pro")

os.makedirs(OUTPUT_DIR, exist_ok=True)

ISSUE_SUMMARY_FILE = os.path.join(OUTPUT_DIR, "formal_specifications_issues.json")


# ==============================
# HELPERS: GLOBAL ISSUE SUMMARY
# ==============================

def append_to_issue_summary(round_number: int, issues: list):
    entry = {
        "round": round_number,
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "issues": issues
    }

    if not os.path.exists(ISSUE_SUMMARY_FILE):
        with open(ISSUE_SUMMARY_FILE, "w", encoding="utf-8") as f:
            json.dump([entry], f, indent=4)
        return

    with open(ISSUE_SUMMARY_FILE, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
            if not isinstance(data, list):
                data = []
        except json.JSONDecodeError:
            data = []

    data.append(entry)

    with open(ISSUE_SUMMARY_FILE, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


def load_issue_summary():
    if not os.path.exists(ISSUE_SUMMARY_FILE):
        return []
    with open(ISSUE_SUMMARY_FILE, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
            return data if isinstance(data, list) else []
        except json.JSONDecodeError:
            return []


# ==============================
# STEP 1 — LOAD JSON INPUT
# ==============================

def load_specs():
    if not os.path.exists(INPUT_FILE):
        raise FileNotFoundError(f"❌ Input file not found: {INPUT_FILE}")

    with open(INPUT_FILE, "r", encoding="utf-8") as f:
        data = json.load(f)

    if not isinstance(data, list):
        raise ValueError("❌ Expected an array of objects in the JSON file.")

    blocks = []

    for idx, entry in enumerate(data, 1):
        fs = entry.get("Formal_Spec", {})
        label = fs.get("LABEL", f"OP_{idx}")
        pre = fs.get("Precondition", "")
        func = fs.get("Function", "")
        post = fs.get("Postcondition", "")

        txt = f"""Operation: {label}
Preconditions:
{pre}

Function:
{func}

Postconditions:
{post}
"""
        blocks.append(txt)

    return "\n------------------------------------\n".join(blocks)


# ==============================
# STEP 2 — GLOBAL UNDECLARED VARIABLE ANALYSIS
# ==============================

def scan_global_variables(spec_text):
    tokens = re.findall(r"[A-Za-z_][A-Za-z0-9_]*", spec_text)

    used = {t for t in set(tokens) if not t.isupper()}

    declared = set()

    match = re.search(r"State Variables(.*?)(Initialization|Operations|$)",
                      spec_text, re.DOTALL | re.IGNORECASE)
    if match:
        section = match.group(1)
        declared = set(re.findall(r"[A-Za-z_][A-Za-z0-9_]*", section))
        declared = {d for d in declared if not d.isupper()}

    keywords = {
        "Operation", "Preconditions", "Function", "Postconditions",
        "If", "Then", "Else", "And", "Or", "Not"
    }

    undeclared = sorted(
        v for v in used
        if v not in declared
        and v not in keywords
        and not v.isupper()
        and not v.startswith("HTTP")
    )

    return undeclared


def detect_issues(spec_text):
    issues = []

    undeclared = scan_global_variables(spec_text)
    if undeclared:
        issues.append(f"Global undeclared variables: {', '.join(undeclared[:20])}")

    return issues


# ==============================
# UTIL: EXTRACT OPERATION LABELS
# ==============================

def extract_operation_labels(text):
    return re.findall(r"Operation:\s*([A-Za-z0-9_]+)", text)


def auto_repair_output(original_text):
    print("🔧 Reinserting missing operations from original...")
    return original_text


# ==============================
# STEP 3 — VERIFY WITH GEMINI (STRICT MODE REMOVED)
# ==============================

def verify_with_gemini(spec_text, round_number, issues):

    expected_ops = extract_operation_labels(spec_text)

    global_issues = load_issue_summary()
    global_issues_json = json.dumps(global_issues, indent=2)

    undeclared_vars = scan_global_variables(spec_text)
    undeclared_txt = ", ".join(undeclared_vars) if undeclared_vars else "None"

    prompt = f"""
You are an assistant that converts natural-language (NL) API documentation into precise, type-safe formal specifications.

The formal specification must always follow this structure:
- Precondition: logical constraints that must hold before execution.
- Function: name, parameters, return type(s), and optional HTTP status code.
- Postcondition: logical constraints on the global state after execution.

You have access to:
1. User-defined types (primitives, maps, sets, tuples).
2. Global state variables (with their types).
3. Function signatures.

Typing rules:
- Γ = variable environment, Δ = function environment.
- Literals: Int, String, Bool.
- Function calls must match Δ signatures.
- Precondition must type-check to Bool.
- Postcondition must type-check using primed globals (X → X′).

Output format must be:

Operation: <LABEL>
Preconditions:
  <logical conditions>

Function:
  <function_name>(...) → <type> [<HTTP>]

Postconditions:
  <logical constraints>

---

ADDITIONAL RULES:

1. Function names DO NOT need to be unique.
   - Reuse existing function names if multiple specs describe similar operations.

2. You MAY remove vague or unusable specs.
   - At the end output:
       REMOVED_SPECS: <count>
       REMOVED_LABELS: [list]

3. Preserve all non-vague specifications.

4. NO Z-notation. No schemas, no Δ(), no ↦, no ∈, no math symbols.

GLOBAL UNDECLARED VARIABLES:
{undeclared_txt}

Detected issues:
{issues}

GLOBAL issue history:
{global_issues_json}

CURRENT SPEC:
---
{spec_text[:11000]}
---

Your task:
- Fix undeclared variables in a State Variables section.
- Reuse existing function names where meaningful.
- Remove vague specs and report them.
"""

    response = MODEL.generate_content(prompt)

    if hasattr(response, "candidates") and response.candidates:
        output = response.candidates[0].content.parts[0].text.strip()
    else:
        output = response.text.strip()

    if not output:
        raise ValueError("Empty LLM output")

    return output


# ==============================
# STEP 4 — VERIFICATION LOOP
# ==============================

def iterative_verification():
    specs = load_specs()
    print("Loaded input specs.\n")

    round_number = 1

    while True:
        print(f"\n================ ROUND {round_number} ================\n")

        issues = detect_issues(specs)
        append_to_issue_summary(round_number, issues)

        print("Issues:", issues)

        verified = verify_with_gemini(specs, round_number, issues)
        specs = verified.strip()

        out_file = os.path.join(OUTPUT_DIR, f"verified_round_{round_number}.txt")
        with open(out_file, "w", encoding="utf-8") as f:
            f.write(specs)
        print(f"Saved: {out_file}")

        new_issues = detect_issues(specs)
        append_to_issue_summary(round_number, new_issues)

        # Stop if fixed
        if not new_issues:
            print("All issues resolved.")
            break

        cont = input("Continue? (y/n): ").strip().lower()
        if cont != "y":
            break

        round_number += 1
        time.sleep(3)

    print("\nFINAL SPEC PREVIEW:\n")
    print(specs[:1000])


# ==============================
# MAIN
# ==============================

if __name__ == "__main__":
    iterative_verification()


Loaded input specs.


================ ROUND 1 ================

Issues: ['Global undeclared variables: Assigned, Complaints, Created, Invitations, Notifications, Orders, Raised, ResetTokens, Sessions, SystemConfig, Under, Warehouse, a, abandoned, accept_solution, access_plumber_feature, accountID, action, activate_plumber, active']
Saved: verification_rounds/verified_round_1.txt

================ ROUND 2 ================

Issues: ['Global undeclared variables: An, Email, PrivilegeLevel, Role, State, UserDetails, address, an, containing, data, defined, fields, for, identifier, information, integer, level, like, of, one']
Saved: verification_rounds/verified_round_2.txt

================ ROUND 3 ================

Issues: ['Global undeclared variables: UserDetails, UserID, and, customer, d, details, dom, email, exists, in, manager, not, plumber, role, signup, u, u_new, where']
Saved: verification_rounds/verified_round_3.txt

================ ROUND 4 ================

Issues: ['Global unde